# Image2Image Pipeline for Stable Diffusion using 🧨 Diffusers 

This notebook shows how to create a custom `diffusers` pipeline for  text-guided image-to-image generation with Stable Diffusion model using  🤗 Hugging Face [🧨 Diffusers library](https://github.com/huggingface/diffusers). 

For a general introduction to the Stable Diffusion model please refer to this [colab](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/stable_diffusion.ipynb).



In [ ]:
!nvidia-smi
!pip install diffusers==0.3.0 transformers ftfy
!pip install -qq "ipywidgets>=7,<8"

You need to accept the model license before downloading or using the weights. In this post we'll use model version `v1-4`, so you'll need to  visit [its card](https://huggingface.co/CompVis/stable-diffusion-v1-4), read the license and tick the checkbox if you agree. 

You have to be a registered user in 🤗 Hugging Face Hub, and you'll also need to use an access token for the code to work. For more information on access tokens, please refer to [this section of the documentation](https://huggingface.co/docs/hub/security-tokens).

In [ ]:
from huggingface_hub import notebook_login
import inspect
import warnings
from typing import List, Optional, Union
import torch
from torch import autocast
from tqdm.auto import tqdm
from diffusers import StableDiffusionImg2ImgPipeline
device = "cuda"
model_path = "CompVis/stable-diffusion-v1-4"

notebook_login()

Load the pipeline

In [ ]:

pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_path,
    revision="fp16", 
    torch_dtype=torch.float16,
    use_auth_token=True
)
img_to_img = pipe.to(device)

import torch
from torch import autocast
from diffusers import StableDiffusionPipeline
text_to_img = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", revision="fp16", torch_dtype=torch.float16, use_auth_token=True)
text_to_img = text_to_img.to("cuda")  

Download an initial image and preprocess it so we can pass it to the pipeline.

In [ ]:
from diffusers import LMSDiscreteScheduler

lms = LMSDiscreteScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear")
pipe.scheduler = lms

In [ ]:
import requests
from io import BytesIO
from PIL import Image

url = "https://i.imgur.com/XCoD24f.jpg"

response = requests.get(url)
init_img = Image.open(BytesIO(response.content)).convert("RGB")
init_img = init_img.resize((768, 512))
init_img

In [ ]:
generator = torch.Generator(device=device).manual_seed(1024)
prompt = "tall and very large magnificient multicolored flower with multicolored shaft, ice pack, two magical multicored igloos in the background, beautiful large stars in the sky. High quality sketch, trending on art station, magical, fairy tale"
with autocast("cuda"):
    image = img_to_img(prompt=prompt, init_image=init_img, strength=0.40, guidance_scale=20, generator=generator).images[0]
image

In [ ]:
history = []
prompts_and_params = []
defaults =  {"seed": 1026, "num_inference_steps":40, "guidance_scale":8}
prompts_and_params.append({
    "prompt": "photorealistic attractive girl, art by monet, sharp details, the girl is an amazingly cute zombie"})

options = {**defaults, **prompts_and_params[-1]}
history.append(options)
generator = torch.Generator("cuda").manual_seed(1025)
with autocast("cuda"):
  image = text_to_img(**options, generator=generator).images[0]

image